In [ ]:
import pywencai
import pandas as pd
import numpy as np
import akshare as ak
from datetime import datetime, timedelta

In [ ]:
start_date = datetime.now() - timedelta(days=120)
start_date_str = start_date.strftime('%Y%m%d')
stock_code='601919'
df = ak.stock_zh_a_hist(
    symbol=stock_code, start_date=start_date_str, adjust='')
df['pct_change'] = df['收盘'].pct_change(periods =1) #计算百分比变化
df['MA_60'] = df['收盘'].rolling(window=60).mean()
df['Above_MA_60'] = (df['收盘'] > df['MA_60']).astype(int)
print(df[["日期",  "开盘", "最高" , "最低" ,  "收盘" ,"MA_60", "pct_change","Above_MA_60"]])


In [ ]:
recent_days = df['pct_change'].tail(13)
max_pct_change_idx = recent_days.idxmax()

max_pct_change_date = df.loc[max_pct_change_idx, '日期']
max_pct_change_date_low = df.loc[max_pct_change_idx, '最低']
max_pct_change_max = df.loc[max_pct_change_idx, 'pct_change']
print(recent_days)
print(max_pct_change_idx,max_pct_change_max)
# 从那一天开始，找到所有后续交易日的最低价
# 使用布尔索引来选择那一天之后的所有日期
mask = df['日期'] >= max_pct_change_date
min_low_after = df.loc[mask, '最低'].min()
print(min_low_after)
# 计算交易日数量
trading_days_after = df.loc[mask]
trading_days_count = trading_days_after.shape[0]
print(min_low_after,trading_days_count)
# 最近一次大涨后，接下来的交易日是否跌穿过大涨当日的最低价
Above_MA_60 = df.loc[df.index[-1], 'Above_MA_60']
print (Above_MA_60,trading_days_count, min_low_after , max_pct_change_date_low, min_low_after > max_pct_change_date_low)

above_support_after_rally = Above_MA_60 == 1 and trading_days_count >3 and trading_days_count<10 and min_low_after > max_pct_change_date_low
print(above_support_after_rally)

In [ ]:
stock_zh_a_hist_min_em_df = ak.stock_zh_a_hist_min_em(symbol="000001", start_date="2024-07-01 09:30:00", end_date="2024-07-01 09:50:00", period="1", adjust="")
print(stock_zh_a_hist_min_em_df)

In [34]:
stock_zh_a_hist_min_em_df = ak.stock_zh_a_hist_min_em(symbol="300486", start_date="2024-07-04 09:15:00", end_date="2024-07-04 09:32:00", period="1", adjust="")
print(stock_zh_a_hist_min_em_df)

                    时间    开盘    收盘    最高    最低    成交量         成交额    最新价
0  2024-07-04 09:30:00  4.97  4.97  4.97  4.97   1961    974617.0  4.970
1  2024-07-04 09:31:00  4.95  5.12  5.13  4.93  15242   7643203.0  5.010
2  2024-07-04 09:32:00  5.07  5.11  5.15  5.07  20381  10418009.0  5.065


In [126]:
stock_zh_a_hist_pre_min_em = ak.stock_zh_a_hist_pre_min_em(symbol="300486", start_time="09:00:00", end_time="09:32:00")
print(stock_zh_a_hist_pre_min_em)

                     时间    开盘    收盘    最高    最低    成交量         成交额    最新价
0   2024-07-05 09:15:00  5.93  5.93  5.93  5.93      0         0.0  5.930
1   2024-07-05 09:16:00  6.30  6.39  6.47  6.30      0         0.0  5.930
2   2024-07-05 09:17:00  6.39  6.24  6.39  6.23      0         0.0  5.930
3   2024-07-05 09:18:00  6.24  6.40  6.40  6.22      0         0.0  5.930
4   2024-07-05 09:19:00  6.40  6.22  6.40  6.22      0         0.0  5.930
5   2024-07-05 09:20:00  6.22  6.20  6.22  6.20      0         0.0  5.930
6   2024-07-05 09:21:00  6.20  6.09  6.20  6.09      0         0.0  5.930
7   2024-07-05 09:22:00  6.09  6.04  6.09  6.04      0         0.0  5.930
8   2024-07-05 09:23:00  6.02  6.02  6.04  6.00      0         0.0  5.930
9   2024-07-05 09:24:00  6.02  5.93  6.02  5.93      0         0.0  5.930
10  2024-07-05 09:25:00  5.93  5.62  5.93  5.62      0         0.0  5.930
11  2024-07-05 09:26:00  5.80  5.80  5.80  5.80  29177  16922660.0  5.800
12  2024-07-05 09:27:00  5.80  5.80  5

In [1]:
import pywencai
import pandas as pd
import numpy as np
import akshare as ak
from datetime import datetime, timedelta

def volume_up_and_price_up(symbol:str):
  df = ak.stock_zh_a_hist_pre_min_em(symbol=symbol, start_time="09:00:00", end_time="09:32:00")
  volume_times =( df.iloc[-2]['成交量'] / df.iloc[-6]['成交量'] )>5
  volume_times_32 =( df.iloc[-1]['成交量'] / df.iloc[-2]['成交量'] )>1.001
  price_times =( df.iloc[-1]['最新价'] / df.iloc[-6]['最新价'] )>1
  # volume_times =( df.iloc[-2]['成交量'] / df.iloc[-6]['成交量'] )>=3
  # volume_times_32 =( df.iloc[-1]['成交量'] / df.iloc[-2]['成交量'] )>1
  # price_times =( df.iloc[-1]['最新价'] / df.iloc[-6]['最新价'] )>=1.001
  # print(volume_times ,  df.iloc[-1]['最新价'] , df.iloc[-6]['最新价'], df.iloc[-1]['最新价'] / df.iloc[-6]['最新价'])
  return volume_times and price_times and volume_times_32

df = ak.stock_zh_a_spot_em()
# 两市剔除ST股、剔除科创板股、剔除北交所股
df = df[~df['代码'].astype(str).str.startswith('4')]
df = df[~df['代码'].astype(str).str.startswith('8')]
df = df[~df['代码'].astype(str).str.startswith('68')]
df = df[~df['名称'].astype(str).str.startswith('N')]
df = df[~df['名称'].astype(str).str.startswith('*')]
df = df[~df['名称'].astype(str).str.startswith('ST')]
# 按成交额降序排序，选出开盘金额最大前N只股
df = df.sort_values(by='成交额', ascending=False)
df = df.head(300)
df['代码'] = df['代码'].astype(str).str[:6]

df['open_up'] = df['今开'] >0.8 * df['昨收']
df = df.where(df['open_up'], np.nan).dropna(
    subset=['open_up'])

# df['量比大于2'] = df['量比'] >= 1.2
# df = df.where(df['量比大于2'], np.nan).dropna(
#     subset=['量比大于2'])

# df['60日涨跌幅大于0'] = df['60日涨跌幅'] > 0
# df = df.where(df['60日涨跌幅大于0'], np.nan).dropna(
#     subset=['60日涨跌幅大于0'])

df['volume_up_and_price_up'] = df['代码'].apply(volume_up_and_price_up)
df = df.where(df['volume_up_and_price_up'], np.nan).dropna(
    subset=['volume_up_and_price_up'])
# print(df[['代码','名称','昨收','今开','最高','最低','最新价','涨跌幅','成交额','量比','振幅']])
# print(df[['代码','名称','昨收','今开','最新价','涨跌幅','成交额','量比','振幅']])
df["ok"] = df['最新价'] > df['今开']
print(df[['代码','名称','ok','昨收','今开','最新价','涨跌幅','成交额']])

          代码     名称     ok      昨收      今开     最新价    涨跌幅           成交额
37    601888   中国中免   True   61.81   62.03   67.99  10.00  3.235784e+09
2957  002475   立讯精密  False   39.09   39.09   38.99  -0.26  1.342507e+09
61    300458   全志科技   True   22.68   22.60   24.42   7.67  8.826639e+08
276   300476   胜宏科技   True   33.38   33.55   34.40   3.06  8.390841e+08
827   000002  万  科Ａ   True    7.13    7.13    7.22   1.26  7.814639e+08
1085  601766   中国中车   True    7.67    7.67    7.74   0.91  7.304380e+08
85    000049   德赛电池   True   24.89   24.80   26.35   5.87  6.726998e+08
2499  002241   歌尔股份   True   19.53   19.52   19.53   0.00  5.928617e+08
1296  002428   云南锗业   True   11.44   11.41   11.52   0.70  5.293283e+08
1543  000725   京东方Ａ   True    4.10    4.09    4.12   0.49  5.049604e+08
555   300346   南大光电   True   28.31   28.10   28.81   1.77  4.974512e+08
1991  300433   蓝思科技   True   17.69   17.70   17.72   0.17  4.128623e+08
1620  600887   伊利股份   True   25.77   25.77   25.88   0.43  3.878

In [48]:
def volume_up_and_price_up(symbol:str):
  df = ak.stock_zh_a_hist_pre_min_em(symbol=symbol, start_time="09:00:00", end_time="09:32:00")
#   print(df.columns)
  volume_times =( df.iloc[-2]['成交量'] / df.iloc[-6]['成交量'] )> 3
  volume_times_32 =( df.iloc[-1]['成交量'] / df.iloc[-2]['成交量'] )>1.001
  price_times =( df.iloc[-1]['最新价'] / df.iloc[-6]['最新价'] )>1.001
  return (volume_times and price_times and volume_times_32 ,
          df.iloc[-3]['开盘'], 
          df.iloc[-1]['收盘'])
# 个股热度前300名
query = f"个股热度前500名"
df = pywencai.get(query=query, query_type="stock")
# print(df.columns)
# 两市剔除ST股、剔除科创板股、剔除北交所股
df = df[~df['股票代码'].astype(str).str.startswith('4')]
df = df[~df['股票代码'].astype(str).str.startswith('8')]
df = df[~df['股票代码'].astype(str).str.startswith('68')]
df = df[~df['股票简称'].astype(str).str.startswith('N')]
df = df[~df['股票简称'].astype(str).str.startswith('*')]
df = df[~df['股票简称'].astype(str).str.startswith('ST')]

df['最新价'] = df['最新价'].astype('float64')
df['最新涨跌幅'] = df['最新涨跌幅'].astype('float64')

df['代码'] = df['股票代码'].astype(str).str[:6]

df['涨跌幅大于-5'] = df['最新涨跌幅'].astype(float) > -3
df = df.where(df['涨跌幅大于-5'], np.nan).dropna(
    subset=['涨跌幅大于-5'])
df['涨跌幅小于11'] = df['最新涨跌幅'].astype(float) < 8
df = df.where(df['涨跌幅大于-5'], np.nan).dropna(
    subset=['涨跌幅小于11'])


df['volume_up_and_price_up'] = df['代码'].apply(volume_up_and_price_up)
df['upup'] = df['volume_up_and_price_up'].map(lambda x: x[0])

df = df.where(df['upup'], np.nan).dropna(
    subset=['upup'])

df['开盘'] = df['volume_up_and_price_up'].map(lambda x: x[1])
df['开盘2分钟价格'] = df['volume_up_and_price_up'].map(lambda x: x[2])
df['开盘2分钟涨幅'] = round(100 * (df['开盘2分钟价格'] - df['开盘']) / df['开盘'],3)
df.sort_values(by='开盘2分钟涨幅', ascending=False, inplace=True)
df[['代码','股票简称','开盘', '开盘2分钟价格','开盘2分钟涨幅','最新价','最新涨跌幅']]


,代码,股票简称,开盘,开盘2分钟价格,开盘2分钟涨幅,最新价,最新涨跌幅
26,301261,恒工精密,34.51,40.74,18.053,40.74,20.000
21,002527,新时达,6.03,6.61,9.619,6.61,9.983
77,300245,天玑科技,5.77,6.07,5.199,6.35,8.547
33,300120,经纬辉开,6.00,6.27,4.500,6.68,10.596
37,300549,优德精密,13.38,13.84,3.438,15.82,20.030
12,300546,雄帝科技,11.31,11.69,3.360,13.72,20.035
44,300386,飞天诚信,9.75,10.07,3.282,10.07,3.922
4,300486,东杰智能,4.97,5.11,2.817,5.93,20.041
78,600719,大连热电,9.08,9.33,2.753,9.30,2.198
32,600476,湘邮科技,12.16,12.47,2.549,13.32,9.992


In [127]:
def volume_up_and_price_up(symbol:str):
  df = ak.stock_zh_a_hist_pre_min_em(symbol=symbol, start_time="09:00:00", end_time="09:32:00")
#   print(df.columns)
  volume_times =( df.iloc[-2]['成交量'] / df.iloc[-6]['成交量'] )> 3
  volume_times_32 =( df.iloc[-1]['成交量'] / df.iloc[-2]['成交量'] )>0.618
  price_times =( df.iloc[-1]['最新价'] / df.iloc[-6]['最新价'] )>1.001
  return (volume_times and price_times and volume_times_32 ,
          df.iloc[-3]['开盘'], 
          df.iloc[-1]['收盘'])
# 个股热度前300名
query = f"个股热度前200名"
df = pywencai.get(query=query, query_type="stock",loop=True)
# print(df.columns)
# 两市剔除ST股、剔除科创板股、剔除北交所股
df = df[~df['股票代码'].astype(str).str.startswith('4')]
df = df[~df['股票代码'].astype(str).str.startswith('8')]
df = df[~df['股票代码'].astype(str).str.startswith('68')]
df = df[~df['股票简称'].astype(str).str.startswith('N')]
df = df[~df['股票简称'].astype(str).str.startswith('*')]
df = df[~df['股票简称'].astype(str).str.startswith('ST')]

df['最新价'] = df['最新价'].astype('float64')
df['最新涨跌幅'] = df['最新涨跌幅'].astype('float64')

df['代码'] = df['股票代码'].astype(str).str[:6]

df['涨跌幅大于-5'] = df['最新涨跌幅'].astype(float) > -3
df = df.where(df['涨跌幅大于-5'], np.nan).dropna(
    subset=['涨跌幅大于-5'])
df['涨跌幅小于11'] = df['最新涨跌幅'].astype(float) < 8
df = df.where(df['涨跌幅大于-5'], np.nan).dropna(
    subset=['涨跌幅小于11'])


df['volume_up_and_price_up'] = df['代码'].apply(volume_up_and_price_up)
df['upup'] = df['volume_up_and_price_up'].map(lambda x: x[0])

df = df.where(df['upup'], np.nan).dropna(
    subset=['upup'])

df['开盘'] = df['volume_up_and_price_up'].map(lambda x: x[1])
df['开盘2分钟价格'] = df['volume_up_and_price_up'].map(lambda x: x[2])
df['开盘2分钟涨幅'] = round(100 * (df['开盘2分钟价格'] - df['开盘']) / df['开盘'],3)
# df.sort_values(by='开盘2分钟涨幅', ascending=False, inplace=True)
# df[['代码','股票简称','开盘', '开盘2分钟价格','开盘2分钟涨幅','最新价','最新涨跌幅']]

df['max_pct'] = df['股票代码'].apply(lambda code: 20 if str(code).startswith('3') else 10)  
df['qd'] = df['开盘2分钟涨幅'] / df['max_pct'] 
df.sort_values(by='qd', ascending=False, inplace=True)

df_filter = df.where(df['开盘2分钟涨幅'] <= 0.8 * df['最新涨跌幅']).dropna()
print(len(df),len(df_filter))
print(df_filter[['代码','股票简称','开盘', '开盘2分钟价格','开盘2分钟涨幅','最新价','最新涨跌幅']])


44 29
         代码   股票简称     开盘  开盘2分钟价格  开盘2分钟涨幅    最新价   最新涨跌幅
73   000890    法尔胜   2.50     2.70    8.000   2.70  10.204
89   300417   南华仪器   8.23     9.49   15.310   9.49  19.975
28   002165    红宝丽   3.48     3.64    4.598   3.75   9.971
22   003005    竞业达  21.97    22.80    3.778  24.51  10.009
9    300391   长药控股   3.70     3.96    7.027   4.62  20.000
90   000679   大连友谊   2.87     2.97    3.484   3.14   4.667
230  603322   超讯通信  28.25    29.02    2.726  29.71   5.168
39   600521   华海药业  16.68    17.04    2.158  17.79  10.019
220  603887   城地香江   5.56     5.68    2.158   5.77   4.340
78   002332   仙琚制药  10.70    10.93    2.150  11.72  10.047
177  603079   圣达生物  11.43    11.66    2.012  11.59   3.022
248  603829   洛凯股份  12.77    12.95    1.410  13.03   2.598
69   300563   神宇股份  24.88    25.53    2.613  27.18   9.244
200  000712   锦龙股份   7.74     7.82    1.034   8.19   6.919
19   601888   中国中免  64.65    65.31    1.021  65.73   1.514
128  600550   保变电气   4.93     4.97    0.811   5.14

In [128]:
def volume_up_and_price_up(symbol:str):
  df = ak.stock_zh_a_hist_pre_min_em(symbol=symbol, start_time="09:00:00", end_time="09:32:00")
#   print(df.columns)
  volume_times =( df.iloc[-2]['成交量'] / df.iloc[-7]['成交量'] )> 3
  volume_times_32 =( df.iloc[-1]['成交量'] / df.iloc[-2]['成交量'] )>0.618
  price_times =( df.iloc[-1]['最新价'] / df.iloc[-7]['最新价'] )>1.001
  return (volume_times and price_times and volume_times_32 ,
          df.iloc[-3]['开盘'], 
          df.iloc[-1]['收盘'])
# 个股热度前300名
query = f"个股热度前200名"
df = pywencai.get(query=query, query_type="stock",loop=True)
# print(df.columns)
# 两市剔除ST股、剔除科创板股、剔除北交所股
df = df[~df['股票代码'].astype(str).str.startswith('4')]
df = df[~df['股票代码'].astype(str).str.startswith('8')]
df = df[~df['股票代码'].astype(str).str.startswith('68')]
df = df[~df['股票简称'].astype(str).str.startswith('N')]
df = df[~df['股票简称'].astype(str).str.startswith('*')]
df = df[~df['股票简称'].astype(str).str.startswith('ST')]

df['最新价'] = df['最新价'].astype('float64')
df['最新涨跌幅'] = df['最新涨跌幅'].astype('float64')

df['代码'] = df['股票代码'].astype(str).str[:6]

df['涨跌幅大于-5'] = df['最新涨跌幅'].astype(float) > -3
df = df.where(df['涨跌幅大于-5'], np.nan).dropna(
    subset=['涨跌幅大于-5'])
df['涨跌幅小于11'] = df['最新涨跌幅'].astype(float) < 8
df = df.where(df['涨跌幅大于-5'], np.nan).dropna(
    subset=['涨跌幅小于11'])


df['volume_up_and_price_up'] = df['代码'].apply(volume_up_and_price_up)
df['upup'] = df['volume_up_and_price_up'].map(lambda x: x[0])

df = df.where(df['upup'], np.nan).dropna(
    subset=['upup'])

df['开盘'] = df['volume_up_and_price_up'].map(lambda x: x[1])
df['开盘2分钟价格'] = df['volume_up_and_price_up'].map(lambda x: x[2])
df['开盘2分钟涨幅'] = round(100 * (df['开盘2分钟价格'] - df['开盘']) / df['开盘'],3)
# df.sort_values(by='开盘2分钟涨幅', ascending=False, inplace=True)
# df[['代码','股票简称','开盘', '开盘2分钟价格','开盘2分钟涨幅','最新价','最新涨跌幅']]

df['max_pct'] = df['股票代码'].apply(lambda code: 20 if str(code).startswith('3') else 10)  
df['qd'] = df['开盘2分钟涨幅'] / df['max_pct'] 
df.sort_values(by='qd', ascending=False, inplace=True)

df_filter = df.where(df['开盘2分钟涨幅'] <= 0.8 * df['最新涨跌幅']).dropna()
print(len(df),len(df_filter))
print(df_filter[['代码','股票简称','开盘', '开盘2分钟价格','开盘2分钟涨幅','最新价','最新涨跌幅']])


26 15
         代码  股票简称     开盘  开盘2分钟价格  开盘2分钟涨幅    最新价   最新涨跌幅
73   000890   法尔胜   2.50     2.70    8.000   2.70  10.204
89   300417  南华仪器   8.23     9.49   15.310   9.49  19.975
28   002165   红宝丽   3.48     3.64    4.598   3.75   9.971
22   003005   竞业达  21.97    22.80    3.778  24.51  10.009
39   600521  华海药业  16.68    17.04    2.158  17.79  10.019
78   002332  仙琚制药  10.70    10.93    2.150  11.72  10.047
177  603079  圣达生物  11.43    11.66    2.012  11.59   3.022
69   300563  神宇股份  24.88    25.53    2.613  27.18   9.244
19   601888  中国中免  64.65    65.31    1.021  65.73   1.514
59   002075  沙钢股份   3.81     3.84    0.787   4.20   9.948
49   002281  光迅科技  36.06    36.34    0.776  37.36   3.864
55   300114  中航电测  47.11    47.82    1.507  49.54   6.859
101  002459  晶澳科技  10.35    10.41    0.580  10.49   1.060
189  300033   同花顺  93.00    93.95    1.022  95.60   2.487
41   002995  天地在线  12.97    12.92   -0.386  14.23   9.969


In [129]:
def volume_up_and_price_up(symbol:str):
  df = ak.stock_zh_a_hist_pre_min_em(symbol=symbol, start_time="09:00:00", end_time="09:32:00")
#   print(df.columns)
  volume_times =( df.iloc[-2]['成交量'] / df.iloc[-7]['成交量'] )> 2
  volume_times_32 =( df.iloc[-1]['成交量'] / df.iloc[-2]['成交量'] )>0.618
  price_times =( df.iloc[-1]['最新价'] / df.iloc[-7]['最新价'] )>1.001
  return (volume_times and price_times and volume_times_32 ,
          df.iloc[-3]['开盘'], 
          df.iloc[-1]['收盘'])
# 个股热度前300名
query = f"个股热度前200名"
df = pywencai.get(query=query, query_type="stock",loop=True)
# print(df.columns)
# 两市剔除ST股、剔除科创板股、剔除北交所股
df = df[~df['股票代码'].astype(str).str.startswith('4')]
df = df[~df['股票代码'].astype(str).str.startswith('8')]
df = df[~df['股票代码'].astype(str).str.startswith('68')]
df = df[~df['股票简称'].astype(str).str.startswith('N')]
df = df[~df['股票简称'].astype(str).str.startswith('*')]
df = df[~df['股票简称'].astype(str).str.startswith('ST')]

df['最新价'] = df['最新价'].astype('float64')
df['最新涨跌幅'] = df['最新涨跌幅'].astype('float64')

df['代码'] = df['股票代码'].astype(str).str[:6]

df['涨跌幅大于-5'] = df['最新涨跌幅'].astype(float) > -3
df = df.where(df['涨跌幅大于-5'], np.nan).dropna(
    subset=['涨跌幅大于-5'])
df['涨跌幅小于11'] = df['最新涨跌幅'].astype(float) < 8
df = df.where(df['涨跌幅大于-5'], np.nan).dropna(
    subset=['涨跌幅小于11'])


df['volume_up_and_price_up'] = df['代码'].apply(volume_up_and_price_up)
df['upup'] = df['volume_up_and_price_up'].map(lambda x: x[0])

df = df.where(df['upup'], np.nan).dropna(
    subset=['upup'])

df['开盘'] = df['volume_up_and_price_up'].map(lambda x: x[1])
df['开盘2分钟价格'] = df['volume_up_and_price_up'].map(lambda x: x[2])
df['开盘2分钟涨幅'] = round(100 * (df['开盘2分钟价格'] - df['开盘']) / df['开盘'],3)
# df.sort_values(by='开盘2分钟涨幅', ascending=False, inplace=True)
# df[['代码','股票简称','开盘', '开盘2分钟价格','开盘2分钟涨幅','最新价','最新涨跌幅']]

df['max_pct'] = df['股票代码'].apply(lambda code: 20 if str(code).startswith('3') else 10)  
df['qd'] = df['开盘2分钟涨幅'] / df['max_pct'] 
df.sort_values(by='qd', ascending=False, inplace=True)

df_filter = df.where(df['开盘2分钟涨幅'] <= 0.8 * df['最新涨跌幅']).dropna()
print(len(df),len(df_filter))
print(df_filter[['代码','股票简称','开盘', '开盘2分钟价格','开盘2分钟涨幅','最新价','最新涨跌幅']])


29 17
         代码  股票简称     开盘  开盘2分钟价格  开盘2分钟涨幅    最新价   最新涨跌幅
73   000890   法尔胜   2.50     2.70    8.000   2.70  10.204
89   300417  南华仪器   8.23     9.49   15.310   9.49  19.975
28   002165   红宝丽   3.48     3.64    4.598   3.75   9.971
22   003005   竞业达  21.97    22.80    3.778  24.51  10.009
9    300391  长药控股   3.70     3.96    7.027   4.62  20.000
39   600521  华海药业  16.68    17.04    2.158  17.79  10.019
78   002332  仙琚制药  10.70    10.93    2.150  11.72  10.047
177  603079  圣达生物  11.43    11.66    2.012  11.59   3.022
69   300563  神宇股份  24.88    25.53    2.613  27.18   9.244
19   601888  中国中免  64.65    65.31    1.021  65.73   1.514
128  600550  保变电气   4.93     4.97    0.811   5.14   4.260
59   002075  沙钢股份   3.81     3.84    0.787   4.20   9.948
49   002281  光迅科技  36.06    36.34    0.776  37.36   3.864
55   300114  中航电测  47.11    47.82    1.507  49.54   6.859
101  002459  晶澳科技  10.35    10.41    0.580  10.49   1.060
189  300033   同花顺  93.00    93.95    1.022  95.60   2.487
41   002

In [112]:
df['max_pct'] = df['股票代码'].apply(lambda code: 20 if str(code).startswith('3') else 10)  
df['qd'] = df['开盘2分钟涨幅'] / df['max_pct'] 
df.sort_values(by='qd', ascending=False, inplace=True)

df_filter = df.where(df['开盘2分钟涨幅'] <= 0.8 * df['最新涨跌幅']).dropna()
print(len(df),len(df_filter))
print(df_filter[['代码','股票简称','开盘', '开盘2分钟价格','开盘2分钟涨幅','最新价','最新涨跌幅']])

df_filter_not = df.where(df['开盘2分钟涨幅'] > 0.8 * df['最新涨跌幅']).dropna()
print(len(df),len(df_filter_not))
print(df_filter_not[['代码','股票简称','开盘', '开盘2分钟价格','开盘2分钟涨幅','最新价','最新涨跌幅']])


40 26
         代码  股票简称     开盘  开盘2分钟价格  开盘2分钟涨幅    最新价   最新涨跌幅
79   000890   法尔胜   2.50     2.70    8.000   2.70  10.204
68   300417  南华仪器   8.23     9.49   15.310   9.49  19.975
42   002165   红宝丽   3.48     3.64    4.598   3.75   9.971
30   003005   竞业达  21.97    22.80    3.778  24.51  10.009
12   300391  长药控股   3.70     3.96    7.027   4.62  20.000
73   000679  大连友谊   2.87     2.97    3.484   3.14   4.667
45   600521  华海药业  16.68    17.04    2.158  17.79  10.019
220  603887  城地香江   5.56     5.68    2.158   5.77   4.340
95   002332  仙琚制药  10.70    10.93    2.150  11.72  10.047
182  603079  圣达生物  11.43    11.66    2.012  11.59   3.022
258  603829  洛凯股份  12.77    12.95    1.410  13.03   2.598
66   300563  神宇股份  24.88    25.53    2.613  27.18   9.244
160  000712  锦龙股份   7.74     7.82    1.034   8.19   6.919
49   601888  中国中免  64.65    65.31    1.021  65.73   1.514
177  600550  保变电气   4.93     4.97    0.811   5.14   4.260
59   002075  沙钢股份   3.81     3.84    0.787   4.20   9.948
130  002

In [133]:
# stock_df = ak.stock_zh_a_hist(symbol='600519', period="daily", start_date='20220101', end_date='20220125', adjust="qfq")
stock_df = ak.stock_zh_a_hist(symbol='600519', period="daily")
stock_df.iloc[-1]

日期       2024-07-08
开盘           1443.0
收盘           1420.0
最高          1457.77
最低          1419.14
成交量           39527
成交额    5657951348.0
振幅             2.66
涨跌幅           -2.27
涨跌额           -33.0
换手率            0.31
Name: 5471, dtype: object

In [142]:
import os
def get_today_ohlc(symbol):
    df = ak.stock_zh_a_hist(symbol=symbol, period="daily", start_date=datetime.now().strftime('%Y%m%d'))
    
    return df.iloc[-1]
def update_today_close_and_pct():
    directory_path = './results'        
    file_name = f"""甘州图灵甄选932_{datetime.now().strftime('%Y%m%d')}.csv"""
    file_path = os.path.join(directory_path, file_name)    
    df = pd.read_csv(file_path,dtype={'代码': 'str'})
    today_trading_data = df['代码'].apply(get_today_ohlc)
    df['收盘'] = today_trading_data['收盘']
    df['涨跌幅'] = today_trading_data['涨跌幅']
    return df
update_today_close_and_pct()

,代码,股票简称,热度排名,开盘,开盘2分钟价格,开盘2分钟涨幅,最新价,最新涨跌幅,open30,volume25,...,volume31,price31,open32,close32,high32,low32,volume32,price32,收盘,涨跌幅
0,000720,新能泰山,57,2.80,2.875,2.679,2.92,10.189,2.80,16377.0,...,71394.0,2.869,2.92,2.92,2.92,2.92,10861.0,2.875,2.67,0.75
1,300264,佳创视讯,156,4.16,4.333,4.159,4.58,17.136,4.16,4313.0,...,36429.0,4.228,4.21,4.68,4.68,4.20,43070.0,4.333,4.19,7.16
2,003037,三和管桩,168,5.97,6.081,1.859,6.05,4.491,5.97,1446.0,...,10275.0,5.960,5.97,6.25,6.31,5.96,13224.0,6.081,5.75,-0.69
3,002600,领益智造,11,6.68,6.800,1.796,6.83,3.799,6.68,117634.0,...,502928.0,6.788,6.83,6.81,6.87,6.78,211784.0,6.800,7.20,9.42
4,600545,卓郎智能,90,1.47,1.495,1.701,1.53,4.795,1.47,35928.0,...,61978.0,1.473,1.50,1.54,1.57,1.50,51324.0,1.495,1.52,4.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,002942,新农股份,107,13.18,12.977,-1.540,12.59,5.004,13.18,5330.0,...,4419.0,13.036,12.78,12.63,12.79,12.61,2063.0,12.977,12.26,2.25
77,002364,中恒电气,70,5.80,5.696,-1.793,5.65,4.824,5.80,7830.0,...,18812.0,5.730,5.65,5.59,5.65,5.57,9478.0,5.696,5.93,10.02
78,603155,新亚强,10,13.40,13.119,-2.097,13.23,5.502,13.40,7679.0,...,21050.0,13.226,12.81,12.90,12.90,12.58,7333.0,13.119,13.79,9.97
79,000422,湖北宜化,12,13.20,12.869,-2.508,12.50,-1.186,13.20,14080.0,...,57168.0,13.000,12.75,12.55,12.75,12.41,30043.0,12.869,12.36,-2.29
